# 评分卡案例
## 建表 导数据
```
set -ue

sql="use tmp_dw_temp; drop table if exists tmp_scorecard_training; drop table if exists tmp_scorecard_test;"
echo "drop table"
hive -e "${sql}"
sql="use tmp_dw_temp;CREATE TABLE if not exists tmp_scorecard_training(  id int COMMENT 'incr id',  SeriousDlqin2yrs int COMMENT 'label',  RevolvingUtilizationOfUnsecuredLines float ,  age int ,  NumberOfTime30_59DaysPastDueNotWorse int,  debtratio float,  monthlyincome int,  numberofopencreditlinesandloans int,  NumberOfTimes90DaysLate int,  numberrealestateloansorlines int,  NumberOfTime60_89DaysPastDueNotWorse int,  NumberOfDependents int  )COMMENT 'score card training data' row format delimited fields terminated by ',' STORED AS textfile;"
hive -e "${sql}"
sql="use tmp_dw_temp;CREATE TABLE if not exists tmp_scorecard_test(  id int COMMENT 'incr id',  SeriousDlqin2yrs int COMMENT 'label',  RevolvingUtilizationOfUnsecuredLines float ,  age int ,  NumberOfTime30_59DaysPastDueNotWorse int,  debtratio float,  monthlyincome int,  numberofopencreditlinesandloans int,  NumberOfTimes90DaysLate int,  numberrealestateloansorlines int,  NumberOfTime60_89DaysPastDueNotWorse int,  NumberOfDependents int  )COMMENT 'score card training data' row format delimited fields terminated by ',' STORED AS textfile;"
hive -e "${sql}"

echo "create tables"

sql="load data inpath '/user/xfjr_bizdata/txu6/ml/scorecard/cs-training.csv' into table tmp_dw_temp.tmp_scorecard_training;"
hive -e "${sql}"
sql="load data inpath '/user/xfjr_bizdata/txu6/ml/scorecard/cs-test.csv' into table tmp_dw_temp.tmp_scorecard_test;"
hive -e "${sql}"
echo "load data"

```

## 导包 配置环境

In [47]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline
import sklearn.metrics as metrics
import os
from datetime import datetime
import math
import time

In [48]:
#配置pyspark的环境参数
#os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6.5"   # jupyter's setting
#os.environ['SPARK_CONF_DIR']='/opt/app/spark/conf'
#os.sys.path.insert(0,'/opt/app/spark/python')
#os.sys.path.insert(1,'/opt/app/spark/python/lib/py4j-0.10.4-src.zip')

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml.feature import Bucketizer
from pyspark.sql import functions as fn

# 初始化SparkSession
idx_type = "mart_Spark_v2"
app_name = datetime.now().strftime('%Y-%m-%d %H:%M:%S') + '_' + idx_type
spark = SparkSession.builder.enableHiveSupport().appName(app_name).config("hive.exec.dynamic.partition",True).config("hive.exec.dynamic.partition.mode","nonstrict").getOrCreate()

## 读取数据

In [51]:

# 利用spark从hive读取数据
df_train = spark.sql("select * from tmp_dw_temp.tmp_scorecard_training").toPandas()
df_test = spark.sql("select * from tmp_dw_temp.tmp_scorecard_test").toPandas()

df_train.head()

,id,seriousdlqin2yrs,revolvingutilizationofunsecuredlines,age,numberoftime30_59dayspastduenotworse,debtratio,monthlyincome,numberofopencreditlinesandloans,numberoftimes90dayslate,numberrealestateloansorlines,numberoftime60_89dayspastduenotworse,numberofdependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [52]:
df_test.head()

,id,seriousdlqin2yrs,revolvingutilizationofunsecuredlines,age,numberoftime30_59dayspastduenotworse,debtratio,monthlyincome,numberofopencreditlinesandloans,numberoftimes90dayslate,numberrealestateloansorlines,numberoftime60_89dayspastduenotworse,numberofdependents
0,1,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,2,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,3,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,4,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,5,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0


## 数据预处理

In [53]:
default_prop = []
for i in range(int(df_train["revolvingutilizationofunsecuredlines"].max())):
    temp_ = df_train.loc[df_train["revolvingutilizationofunsecuredlines"] >= i]
    default_prop.append([i, temp_["seriousdlqin2yrs"].mean()])

late_pay_cols = ["numberoftimes90dayslate", "numberoftime60_89dayspastduenotworse",
                 "numberoftime30_59dayspastduenotworse"]

distinct_triples_counts = dict()
for arr in df_train.loc[df_train["numberoftimes90dayslate"] > 17][late_pay_cols].values:
    triple = ",".join(list(map(str, arr)))
    if triple not in distinct_triples_counts:
        distinct_triples_counts[triple] = 0
    else:
        distinct_triples_counts[triple] += 1


## 特征筛选

In [54]:

# 中位数填充NA
df_train = df_train.fillna(df_train.median())
# 分箱
age_bins = [-math.inf, 25, 40, 50, 60, 70, math.inf]
df_train['bin_age'] = pd.cut(df_train['age'], bins=age_bins).astype(str)
dependent_bin = [-math.inf, 2, 4, 6, 8, 10, math.inf]
df_train['bin_numberofdependents'] = pd.cut(df_train['numberofdependents'], bins=dependent_bin).astype(str)
dpd_bins = [-math.inf, 1, 2, 3, 4, 5, 6, 7, 8, 9, math.inf]
df_train['bin_numberoftimes90dayslate'] = pd.cut(df_train['numberoftimes90dayslate'], bins=dpd_bins)
df_train['bin_numberoftime30_59dayspastduenotworse'] = pd.cut(df_train['numberoftime30_59dayspastduenotworse'],
                                                              bins=dpd_bins)
df_train['bin_numberoftime60_89dayspastduenotworse'] = pd.cut(df_train['numberoftime60_89dayspastduenotworse'],
                                                              bins=dpd_bins)

df_train['bin_revolvingutilizationofunsecuredlines'] = pd.qcut(df_train['revolvingutilizationofunsecuredlines'], q=5,
                                                               duplicates='drop').astype(str)
df_train['bin_debtratio'] = pd.qcut(df_train['debtratio'], q=5, duplicates='drop').astype(str)
df_train['bin_monthlyincome'] = pd.qcut(df_train['monthlyincome'], q=5, duplicates='drop').astype(str)
df_train['bin_numberofopencreditlinesandloans'] = pd.qcut(df_train['numberofopencreditlinesandloans'], q=5,
                                                          duplicates='drop').astype(str)
df_train['bin_numberrealestateloansorlines'] = pd.qcut(df_train['numberrealestateloansorlines'], q=5,
                                                       duplicates='drop').astype(str)

bin_cols = [c for c in df_train.columns.values if c.startswith('bin_')]


# 特征筛选  iv计算
def cal_IV(df, feature, target):
    lst = []
    cols = ['Variable', 'Value', 'All', 'Bad']
    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature, val, df[df[feature] == val].count()[feature],
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]])
    #     print(lst)
    data = pd.DataFrame(lst, columns=cols)
    data = data[data['Bad'] > 0]

    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Bad'] / data['Distribution Good'])
    data['IV'] = (data['WoE'] * (data['Distribution Bad'] - data['Distribution Good'])).sum()

    data = data.sort_values(by=['Variable', 'Value'], ascending=True)

    return data['IV'].values[0]


def cal_WOE(df, features, target):
    df_new = df
    for f in features:
        df_woe = df_new.groupby(f).agg({target: ['sum', 'count']})
        df_woe.columns = list(map(''.join, df_woe.columns.values))
        df_woe = df_woe.reset_index()
        df_woe = df_woe.rename(columns={target + 'sum': 'bad'})
        df_woe = df_woe.rename(columns={target + 'count': 'all'})
        df_woe['good'] = df_woe['all'] - df_woe['bad']
        df_woe = df_woe[[f, 'good', 'bad']]
        df_woe['bad_rate'] = df_woe['bad'].mask(df_woe['bad'] == 0, 1) / df_woe[
            'bad'].sum()  # mask 0 to 1 to avoid log(0)
        df_woe['good_rate'] = df_woe['good'] / df_woe['good'].sum()

        df_woe['woe'] = np.log(df_woe['bad_rate'].divide(df_woe['good_rate'], fill_value=1))
        df_woe.columns = [c if c == f else c + '_' + f for c in list(df_woe.columns.values)]
        df_new = df_new.merge(df_woe, on=f, how='left')
    return df_new


# 我们根据IV选出来的特征
feature_cols = ['revolvingutilizationofunsecuredlines', 'numberoftime30_59dayspastduenotworse', 'age',
                'numberoftimes90dayslate', 'numberoftime60_89dayspastduenotworse']
bin_cols = ['bin_revolvingutilizationofunsecuredlines', 'bin_numberoftime30_59dayspastduenotworse', 'bin_age',
            'bin_numberoftimes90dayslate', 'bin_numberoftime60_89dayspastduenotworse']
df_woe = cal_WOE(df_train, bin_cols, 'seriousdlqin2yrs')
woe_cols = [c for c in list(df_woe.columns.values) if 'woe' in c]

df_bin_to_woe = pd.DataFrame(columns=['features', 'bin', 'woe'])
for f in feature_cols:
    b = 'bin_' + f
    w = 'woe_bin_' + f
    df = df_woe[[w, b]].drop_duplicates()
    df.columns = ['woe', 'bin']
    df['features'] = f
    df = df[['features', 'bin', 'woe']]
    df_bin_to_woe = pd.concat([df_bin_to_woe, df])

print("done")

done


## 模型训练

In [56]:

# 划分训练集 测试集
X_train, X_test, y_train, y_test = train_test_split(df_woe[woe_cols], df_woe['seriousdlqin2yrs'], test_size=0.2,
                                                    random_state=42)

# 训练逻辑回归
model = LogisticRegression(random_state=42).fit(X_train, y_train)

# 打分
print(model.score(X_test, y_test))

# 混淆矩阵
y_pred = model.predict(X_test)
print(metrics.confusion_matrix(y_test, y_pred))

0.9370333333333334
[[27888   156]
 [ 1733   223]]


## 评分卡转换

In [57]:

# 评分卡转换
A = 650
B = 72.13


def generate_scorecard(model_coef, binning_df, features, B):
    lst = []
    cols = ['Variable', 'Binning', 'Score']
    coef = model_coef[0]
    for i in range(len(features)):
        f = features[i]
        df = binning_df[binning_df['features'] == f]
        for index, row in df.iterrows():
            lst.append([f, row['bin'], int(round(-coef[i] * row['woe'] * B))])
    data = pd.DataFrame(lst, columns=cols)
    return data


score_card = generate_scorecard(model.coef_, df_bin_to_woe, feature_cols, B)
print(score_card)


                                Variable           Binning  Score
0   revolvingutilizationofunsecuredlines  (0.699, 50708.0]    -68
1   revolvingutilizationofunsecuredlines    (0.271, 0.699]     -3
2   revolvingutilizationofunsecuredlines   (0.0832, 0.271]     48
3   revolvingutilizationofunsecuredlines  (-0.001, 0.0192]     71
4   revolvingutilizationofunsecuredlines  (0.0192, 0.0832]     80
5   numberoftime30_59dayspastduenotworse        (1.0, 2.0]    -70
6   numberoftime30_59dayspastduenotworse       (-inf, 1.0]     11
7   numberoftime30_59dayspastduenotworse        (2.0, 3.0]    -88
8   numberoftime30_59dayspastduenotworse        (3.0, 4.0]   -101
9   numberoftime30_59dayspastduenotworse        (4.0, 5.0]   -105
10  numberoftime30_59dayspastduenotworse        (6.0, 7.0]   -117
11  numberoftime30_59dayspastduenotworse        (9.0, inf]   -123
12  numberoftime30_59dayspastduenotworse        (5.0, 6.0]   -119
13  numberoftime30_59dayspastduenotworse        (7.0, 8.0]    -81
14  number

In [58]:
# 分组排序显示
sort_scorecard = score_card.groupby('Variable').apply(lambda x: x.sort_values('Score', ascending=False))
print(sort_scorecard)

                                                                     Variable  \
Variable                                                                        
age                                  17                                   age   
                                     19                                   age   
                                     18                                   age   
                                     15                                   age   
                                     16                                   age   
                                     20                                   age   
numberoftime30_59dayspastduenotworse 6   numberoftime30_59dayspastduenotworse   
                                     5   numberoftime30_59dayspastduenotworse   
                                     13  numberoftime30_59dayspastduenotworse   
                                     14  numberoftime30_59dayspastduenotworse   
                            